#### 301. Remove Invalid Parentheses
* https://leetcode.com/problems/remove-invalid-parentheses/description/
#### BBG

In [ ]:
# Deep dive - https://chatgpt.com/c/695638a2-2ab0-8323-9fe9-bce9365a70cf
# Preferred solution - BFS using Queue + Set
# TC - 2^n, SC - 2^n

from typing import List, Set, Deque
from collections import deque

class Solution:
    """
    Preferred solution - BFS using Queue + Set
    TC - 2^n, SC - 2^n
    
    Pruning - In practice, it is much less as we find a solution at the given level once
    we don't go further deep in the tree as all solution will be on that level
    of the tree. This is achieved using 'found' variable 

    Set helps to avoid compute on the previously traversed substring
    
    """
    def removeInvalidParentheses(self, s: str) -> List[str]:
        def is_valid(st: str) -> bool:
            balance_counter: int = 0 
            for c in st:
                if c == '(':
                    balance_counter += 1
                elif c == ')':
                    balance_counter -= 1
                    if balance_counter < 0:
                        return False
            return balance_counter == 0

        parentheses_set: Set[int] = set([s])
        parentheses_queue: Deque[int] = deque([s])
        # once a level is found we don't wont to go deeper
        found = False
        res = []
        while parentheses_queue:
            curr_str = parentheses_queue.popleft()
            if is_valid(curr_str):
                res.append(curr_str)
                found = True

            if found:
                continue

            for i in range(len(curr_str)):
                if curr_str[i] not in '()':
                    continue
                next_str: str = curr_str[:i] + curr_str[i+1:]
                if next_str not in parentheses_set:
                    parentheses_set.add(next_str)
                    parentheses_queue.append(next_str)
        
        return res



In [ ]:
# Deep dive - https://chatgpt.com/c/695638a2-2ab0-8323-9fe9-bce9365a70cf
# Preferred solution - BFS using Queue + Set
# TC - 2^n, SC - 2^n

from typing import List
from collections import deque

class Solution:
    def removeInvalidParentheses(self, s: str) -> List[str]:
        # Preferred solution - BFS using Queue + Set
        # TC - 2^n, SC - 2^n
        
        # Pruning - In practice, it is much less as we find a solution at the given level once
        # we don't go further deep in the tree as all solution will be on that level
        # of the tree. This is achieved using 'found' variable 

        # Set helps to avoid compute on the previously traversed substring


        def is_valid(st: str) -> bool:
            '''Check if the input string is valid i.e balanced parentheses'''
            balance = 0
            for char in st:
                if char == '(':
                    balance += 1
                elif char == ')':
                    balance -= 1
                    if balance < 0:
                        return False
            return balance == 0

        visited = set([s])
        q = deque([s])
        found = False # most imp variable
        res = []
        while q:
            curr_str = q.popleft()
            if is_valid(curr_str):
                res.append(curr_str)
                found = True

            # Pruning logic - don't go any level deeper in the tree
            if found:
                continue

            # Eliminate ith char from the string
            for i in range(len(curr_str)):
                if curr_str[i] not in '()':
                    continue
                nxt = curr_str[:i] + curr_str[i+1:]
                if nxt not in visited:
                    visited.add(nxt)
                    q.append(nxt)
        
        return res


In [ ]:
# TC, SC - O(2^n), O(n)

from typing import List
class Solution:
    def removeInvalidParentheses(self, s: str) -> List[str]:
        def _get_min_removal():
            '''number of required left ')' and right ')' removal to balance the parentheses'''
            lr, rr = 0, 0
            for c in s:
                if c == '(':
                    lr += 1
                elif c == ')':
                    if lr > 0:
                        lr -= 1
                    else:
                        rr += 1
            return lr, rr

        lr, rr = _get_min_removal()
        n = len(s)

        res = set() # don't allow duplicates
        sol = []

        def bt(i, balance, lr, rr):
            # negative case
            if balance < 0 or lr < 0 or rr < 0:
                return

            # positive case
            if i == n:
                if balance == 0 and lr == 0 and rr == 0:
                    res.add(''.join(sol))
                return

            c = s[i]
            # Option 1 - Remove braces and check if balanced
            if c == '(' and lr > 0:
                bt(i+1, balance, lr-1, rr)
            elif c == ')' and rr > 0:
                bt(i+1, balance, lr, rr-1)

            
            # Option 2 - Keep braces and see if balanced
            sol.append(c)
            if c not in '()':
                bt(i+1, balance, lr, rr)
            elif c == '(': # balance counts the number of open braces '('
                bt(i+1, balance+1, lr, rr)
            elif c == ')':
                bt(i+1, balance-1, lr, rr)
            sol.pop()

        bt(0, 0, lr, rr)
        return list(res)
        

In [10]:
class Solution:
    def minDistance(self, houses, k: int) -> int:
        # 2 Steps
        # PreCompute min distance cost by summation of abs diff bw house and median for houses between i and j 
        # using median - eg if houses are 1, 4, 8 then median distance is 7 better than avg or last element etc - L1 Norm for Optimization - Global minima
        # Think like we are trying to create clusters of homes by partitoning and placing post office in those clusters

        # Create k+1*n+1 dp
        # i is 1 to n+1, j is from 0 to i
        # m is from 1 to k+1
        # think inversely
        # at each step of dp[m][i] we want the min distance of dp[m][i] and
        # dp[m-1][j] + cost[j][i-1] # consider it like we have 0-j houses left after this steps
        # this steps uses 1 postoffice between j and i-1 hence cost[j][i-1]
        # now we are left with m-1 post offices to cover from 0-j houses

        # Step 1 - Compute min distance cost from houses bw i and j using median and summation of distance 
        n = len(houses)
        houses.sort() # nlogn 

        cost = [[0]*n for _ in range(n)]
        for i in range(n):
            for j in range(i, n):
                m = (i+j)//2
                cost[i][j] = sum(abs(houses[x]-houses[m]) for x in range(i, j+1))

        print(cost)
        # Step 2 -
        # Bottom up dp for min distance
        dp = [[float('inf')]*(n+1) for _ in range(k+1)]
        dp[0][0] = 0 # 0 post offices for 0 house

        for m in range(1, k+1): # 1, k+1 to support dp
            for i in range(1, n+1):
                for j in range(i):
                    dp[m][i] = min(dp[m][i], dp[m-1][j] + cost[j][i-1])
        
        return dp[k][n]
    
Solution().minDistance([2,3,5,12,18], k = 2)


[[0, 1, 3, 12, 25], [0, 0, 2, 9, 22], [0, 0, 0, 7, 13], [0, 0, 0, 0, 6], [0, 0, 0, 0, 0]]


9

In [3]:
1//0

ZeroDivisionError: integer division or modulo by zero